In [1]:
import numpy as np
import pandas as pd

from dask.distributed import Client

from condor_utils import make_htcondor_cluster

In [2]:
cluster = make_htcondor_cluster(local=False, dashboard_address=13345)
cluster

In [3]:
cluster.scale(5)

In [4]:
client = Client(cluster)
client

Client Scheduler: tcp://169.228.130.74:8748 Dashboard: http://169.228.130.74:13345/status,Cluster Workers: 3 Cores: 3 Memory: 6.00 GB


In [7]:
import time
import random
futures = client.map(lambda x:time.sleep(0.2*random.random()), range(50))
results = client.gather(futures)